In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import skimage
import time

In [ ]:
def preProcessFromPath(image, listImg, listAns, category):
    assert type(image) != None
    assert image.shape == (1080, 1920, 3)
    image = cv2.resize(image, (28, 28))
    assert type(image) == np.ndarray
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.astype('float32')
    image = image/255.0
    listImg.append(image)
    listAns.append(category)
    return listImg, listAns

def captureImgWithPath(path, category):
    fileList = os.listdir(path)
    imgList = []
    ansList = []
    for name in fileList:
        image_full_path = path + '/' + name
        img = cv2.imread(image_full_path)
        imgList, ansList = preProcessFromPath(img, imgList, ansList, category)
    imgArr = np.array(imgList)
    return imgArr, ansList

def captureImgWithCam(img):
    if img is not None:
        imgList, _ = preProcessFromPath(img, [], [], 0)
        imgArr = np.array(imgList)
        return imgArr
    else:
        assert(0)
    return None

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def computeNextState(listMsd, previousState, threshold):
    if len(listMsd) > 4:
        listMsd = listMsd[1 : ]
    avg = sum(listMsd)/len(listMsd)
    if previousState == 'empty' and avg > threshold:
        return 'inMotionForward', listMsd
    elif previousState == 'inMotionForward' and avg < threshold:
        return 'filled', listMsd
    elif previousState == 'filled' and avg > threshold:
        return 'inMotionBackward', listMsd
    elif previousState == 'inMotionBackward' and avg < threshold:
        return 'empty', listMsd
    else:
        return previousState, listMsd

In [ ]:
#paths for all of the images

cup_train_path = '/Users/23amrutad/Desktop/data/cup-train-standing'
cup_test_path = '/Users/23amrutad/Desktop/data/cup-test'
apple_train_path = '/Users/23amrutad/Desktop/data/apple-train'
apple_test_path = '/Users/23amrutad/Desktop/data/apple-test'

#
kLabelCup = 0
kLabelApple = 1

#lists to put the images and corresponding answers into for train and test
cupImg, cupAns = captureImgWithPath(cup_train_path, kLabelCup)
appleImg, appleAns = captureImgWithPath(apple_train_path, kLabelApple)

ImagesTrain = np.vstack((cupImg, appleImg))
AnswersTrain = cupAns + appleAns
AnswersTrain = np.array(AnswersTrain)
print('ImagesTrain shape is ', ImagesTrain.shape)
print('AnswersTrain shape is ', AnswersTrain.shape)


cupImg, cupAns = captureImgWithPath(cup_test_path, kLabelCup)
appleImg, appleAns = captureImgWithPath(apple_test_path, kLabelApple)
ImagesTest = np.vstack((cupImg, appleImg))
AnswersTest = cupAns + appleAns
AnswersTest = np.array(AnswersTest)
print('ImageTest shape is ', ImagesTest.shape)
print('AnswersTest shape is ', AnswersTest.shape)


x_train, y_train = unison_shuffled_copies(ImagesTrain, AnswersTrain)
x_test = ImagesTest
y_test = AnswersTest

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
 
input_shape = (28, 28, 1)

 
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(16, activation=tf.nn.relu))
model.add(Dropout(0.5))
model.add(Dense(2, activation=tf.nn.softmax))
 
 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=60)
model.evaluate(x_test, y_test)

In [ ]:
previousState = 'empty'
previousImg = None
currentImg = None
cap = cv2.VideoCapture(0)
msdList = []

while(True):
    time.sleep(0.1)
    ret, currentImg = cap.read()
    if (previousImg is not None):
        msd = skimage.measure.compare_mse(currentImg, previousImg)
        msdList.append(msd)
        state, msdList = computeNextState(msdList, previousState, 500)
        #print('the state is ', state)
        if previousState != 'filled' and state == 'filled':
            #print(currentImg.shape, ' is the image shape')
            imgList, _ = preProcessFromPath(currentImg, [], [], 0)
            imgArr = np.array(imgList)
            imgArr = imgArr.reshape(imgArr.shape[0], 28, 28, 1)
            prediction = model.predict(imgArr)
            predictionList = prediction.tolist()
            if predictionList[0][0] > predictionList[0][1]:
                print('it is a cup ', predictionList)
            elif predictionList[0][1] > predictionList[0][0]:
                print('it is an apple ', predictionList)
            else:
                print('confused ', predictionList)
            print(predictionList[0][0])
        previousState = state
    previousImg = currentImg